<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/Agent_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#building agents from scratch .by god's grace. Jai Shri Ram . Jai Bajrangbali
#building agents using MCP Protcol
#extending the agent to react and using memory in this -> using langmem sdk from langrapgh -> 1102
#https://www.youtube.com/watch?v=aHCDrAbH_go&t=120s - LangGraph
#https://www.analyticsvidhya.com/blog/2025/03/langmem-sdk/ - Analytics Vidhya
#https://docs.langchain.com/oss/python/langgraph/add-memory#add-short-term-memory
#https://medium.com/@devwithll/simple-langgraph-implementation-with-memory-asyncsqlitesaver-checkpointer-fastapi-54f4e4879a2e - CODE FOR RUNNING CONV AI USING GRAPH
!pip install langchain_core langchain-anthropic langgraph

In [ ]:
#import libraries
import os ,getpass
def _set_env(var:str):
    os.environ[var] = getpass.getpass(f"Enter your {var}: ")
_set_env("ANTHROPIC_API_KEY")

In [ ]:
#llm
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model="claude-3-5-sonnet-latest")


In [ ]:
#schema for structured output
from pydantic import BaseModel , Field
class SearchQuery(BaseModel):
    search_query: str = Field(None , description="Query that is optimized web search.")
    justification: str = Field(None , description="Why this query is relevant to user's request.")

#Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

#invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high chlorestrol levels?")
print(output.search_query)
print(output.justification)

In [ ]:
#tool calling
def multiply(a:int,b:int) ->int:
  return a*b
#augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])
#invoke the LLM with input that triggers the tool call
msg = llm_with_tools.invoke("What is 2 times 3?")
print(msg)

#get the tool call
msg.tool_calls

In [ ]:
#prompt chaining
#each llm processes the output of the other llm in a chain
#when do you want to do this
#use it in Conv AI .... in LLM-as-a-judge
#!pip install typing_extensions
from typing_extensions import TypedDict
#graph state for passing thru one llm to another llm. this is KEY
class State(TypedDict):
  topic: str
  joke: str
  improved_joke: str
  final_joke: str

In [ ]:
#Nodes
def generate_joke(state:State):
  """First LLM call to generate initial joke"""
  msg = llm.invoke(f"Write a short joke {state('topic')}")
  return {"joke":msg.content}

def improve_joke(state:State):
  """Second LLM call to improve the joke"""
  msg = llm.invoke(f"Make this joke funnier by adding wordplay{state('joke')}")
  return {"improved_joke":msg.content}

def polish_joke(state:State):
  """Third LLM call for final polish of the joke content to make it better"""
  msg = llm.invoke(f"Add a surprising twist to this joke{state('joke')}")
  return {"final_joke": msg.content}

#conditional edge function to check if the joke has a punchline
def check_punchline(state:State): #conditional EDGE In Langgraph - what is the condition to move from one llm to another
    """Gate Function to check if the joke has a punchline"""
    if "?" in state["improved_joke"] or "!" in state["joke"]:
      return "Pass"
    return "Fail"

In [ ]:
#langgraph simple workflow
from langgraph.graph import StateGraph , START , END
from IPython.display import Image, display

#build workflow
workflow = StateGraph(State)
workflow.add_node(generate_joke, generate_joke)
workflow.add_node(improve_joke,improve_joke)
workflow.add_node(polish_joke,polish_joke)

#add edges to connect nodes
workflow.add_edge(START,"generate_joke")
workflow.add_conditional_edges("generate_joke",check_punchline , {"Pass":"improve_joke", "Fail":END})
workflow.add_edge("improve_joke","polish_joke")
workflow.add_edge("polish_joke",END)

#compile workflow
chain = workflow.compile()

#show workflow
display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
#
state = chain.invoke({"topic":"cats"})
print("Intial Joke:")
print(state["joke"])
print("\n=== === ===\n")
if "improved_joke" in state:
  print("Improved Joke:")
  print(state["improved_joke"])
  print("Final Joke:")
  print(state["final_joke"])
else:
  print("joke failed quality gate = no punchline detected")

In [ ]:
#parallelization
#one task and you fan out to sub task to separate LLMs to parallelize the task and do a final aggregration of the task
#Graph State
class State(TypedDict):
  topic:str
  joke: str
  story: str
  poem:str
  combined_output: str

In [ ]:
#nodes
def call_llm_1(state:State):
  """ Fist LLM call to generate intial joke """
  msg = llm.invoke(f"Write a short joke about {state('topic')}")
  return {"joke":msg.content}

def call_llm_2(state:State):
  """ Second LLM call to generate Story"""
  msg = llm.invoke(f"Write a short story about {state('topic')}")
  return {"story":msg.content}

def call_llm_3(state:State):
  """Third LLM call to generate Poem"""
  msg = llm.invoke(f"Write a short poem about {state('topic')}")
  return {"poem":msg.content}

def aggregator(state:State):
  """ Combine the joke and story into a single output"""
  combined = f"Here's a story , joke and poem about {state['topic']}!\n\n"
  combined += f"Joke: {state['joke']}\n\n"
  combined += f"Story: {state['story']}\n\n"
  combined += f"Poem: {state['poem']}"
  return {"combined_output":combined}

In [ ]:
#build parallel workflow
parallel_builder = StateGraph(State)

#add nodes
parallel_builder.add_node("call_llm_1",call_llm_1)
parallel_builder.add_node("call_llm_2",call_llm_2)
parallel_builder.add_node("call_llm_3",call_llm_3)
parallel_builder.add_node("aggregator",aggregator)

#add edges to connect nodes
parallel_builder.add_edge(START,"call_llm_1")
parallel_builder.add_edge(START,"call_llm_2")
parallel_builder.add_edge(START,"call_llm_3")
parallel_builder.add_edge("call_llm_1","aggregator")
parallel_builder.add_edge("call_llm_2","aggregator")
parallel_builder.add_edge("call_llm_3","aggregator")
parallel_builder.add_edge("aggregator",END)
parallel_workflow = parallel_builder.compile()

#show workflow
display(Image(parallel_workflow.get_graph().draw_mermaid_png()))

In [ ]:
#routing
#take a input and route the input to poem, story or joke generation based on user input and then aggregate
from typing_extensions import Literal
class Route(BaseModel):
  step:Literal["poem","story","joke"] = Field(None , description = "The next step in the routing process")
#augment the LLM with schema for structured output
router = llm.with_structured_output(Route)

In [ ]:
#state for routing
#one task and you fan out to sub task to separate LLMs to parallelize the task and do a final aggregration of the task
#Graph State
class State(TypedDict):
  input:str
  decision: str
  output: str

In [ ]:
#
from langchain_core.messages import HumanMessage , SystemMessage

def call_llm_1(state:State):
  """ Write a story """
  result = llm.invoke(f"Write a short story about {state('input')}")
  return {"output":result.content}

def call_llm_2(state:State):
  """ Write a Joke """
  result = llm.invoke(f"Write a short joke  about {state('input')}")
  return {"output":result.content}

def call_llm_3(state:State):
  """ Write a Poem  """
  result = llm.invoke(f"Write a short poem  about {state('input')}")
  return {"output":result.content}

def llm_call_router(state:State):
  """ Route the input to the appropiate node """
  #run the augmented LLM with structured output to serve as routing logic
  decision = router.invoke([SystemMessage(content = "Route the input to Story , Joke or Poem based on the user's request."),
                            HumanMessage(content = state("input"))])
  return {"decision": decision.step}

#conditional edge function to route to the appropiate node #dotted line show conditional edge
def route_decision(state:State):
  #return the node name you want to visit next
  if state["decision"] == "story":
    return call_llm_1
  elif state["decision"] == "joke":
    return call_llm_2
  elif state["decision"] == "poem":
    return call_llm_3
  else:
    return

In [ ]:
#build parallel workflow
router_builder = StateGraph(State)

#add nodes
router_builder.add_node("call_llm_1",call_llm_1)
router_builder.add_node("call_llm_2",call_llm_2)
router_builder.add_node("call_llm_3",call_llm_3)
#router_builder.add_node("aggregator",aggregator)

#add edges to connect nodes
router_builder.add_edge("call_llm_1",END)
router_builder.add_edge("call_llm_2",END)
router_builder.add_edge("call_llm_3",END)

router_workflow = router_builder.compile()
#show workflow
display(Image(router_builder.get_graph().draw_mermaid_png()))

In [ ]:
#Orchestrator-Worker - LLM breaks down a task and delegate each task to a worker and synthensize to provide outcome
from typing import Annotated , List
import operator
#schema for structure output to use in planning


In [ ]:
#Evaluator-optimizer workflow
#one LLM generates a response while another LLM evaluates and provide feedback in loop
#schema for structured output to use in evaluation
class Feedback(BaseModel):
  grade:Literal["funny","not funny"] = Field(description = "Decide if the joke is funny or not",)
  feedback :str = Field(description = "Explain why the joke is funny or not")

#augment the llm with schema for structured output
evaluator = llm.with_structured_output(Feedback)

In [ ]:
#Graph State
class State(TypedDict):
  joke:str
  topic:str
  feedback:str
  funny_or_not:str

In [ ]:
#Nodes
def llm_call_generator(state:State):
  """LLM generate a joke"""
  if state.get("feedback"):
    msg = llm.invoke(f"Write a joke about {state ['topic']} but take this feedback into account {state['feedback']}")
  else:
    msg = llm.invoke(f"Write a joke about {state ['topic']}")
  return {"joke":msg.content}

def llm_call_evaluator(state:State):
  """ LLM evaluates a joke """
  grade = evaluator.invoke(f"Grade the joke{state['joke']}")
  return {"funny_or_not":grade.grade , "feedback":grade.feedback}

#conditional edge function to route back to joke generator or end based upon feedback from the evaluator
def route_joke():
  """ Route the joke back to the joke generator or end based upon feedback from the evaluator"""
  if state["funny_or_not"] == "funny":
    return "Accepted"
  elif state["funny_or_not"] == "not funny":
    return "Rejected + Feedback"

In [ ]:
#Agent
#remove scafolding and allow LLM to take actions
#define tools using tool decorator
#agent
from langchain_core.tools import tool

@tool
def multiply(a:int , b:int) ->int:
  return a*b

@tool
def add(a:int , b:int) ->int:
  return a+b

@tool
def divide(a:int , b:int) ->int:
  return a/b

@tool
def subtract(a:int , b:int) ->int:
  return a-b

#augment the llm with tools
tools = [add, multiply, divide, subtract]
tools_by_name = {tool.name:tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)


In [ ]:
#
from langgraph.graph import MessagesState
from langchain_core.messages import ToolMessage
from IPython.display import Image, display

#Nodes
def llm_call(state:MessagesState):
  """LLM decides whether to call a tool or not"""
  return {"messages":[llm_with_tools.invoke([SystemMessage(content= "You are a helpful Assistant tasked with performing arithmetic on a set of inputs")]+ state["messages"])]}

def tool_node(state:dict):
  """Call a tool"""
  result = []
  for tool_call in state["messages"][-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    result.append(ToolMessage(content = observation , tool_call_id = tool_call["id"]))
  return {"messages":result}

def should_continue(state:MessagesState) -> Literal["enviornment":"END"]:
  """ decide if we should continue the loop or stop based on upon whether the LLM made a tool call"""
  messages = state["messages"]
  last_message = messages[-1]
  if last_message.tool_calls:
        return "Action"
  return "END"


CODE OF USING PERSISTENCE MEMORY EITH REACT AGENT USING TOOLS AND ALSO STORING RETREIVING MEMORY
* https://docs.langchain.com/oss/python/langgraph/add-memory#add-short-term-memory


*   Checkpoints need to be used backed by Persistence Store like a Redis or Postgress of Mongodb
*  All memory has to be stored in the persistence memory using a session_id
*  These memory can be searched . Memory can be of a Conversational Memory and stored in this system at the end of the conversation   
*   Langgraph uses State object for passing memory for in-session
*   Langgraph uses Checkpoints for persistence memory and these can be passed to graph nodes or sub-agents
*   Tools can be binded to model. USE CREATE_REACT_AGENT
*   Memory Management should be Async when handling production grade Conv Agents
*   Handle Rate Limits with foundational models
*   Error handling try :except block with proper handling of error should be there












In [ ]:
#start code for React Agent using LT memory ....
#ST Memory - In session can be handled via State but LT memory we need mem0 or something
#we will build the code using mem0 also
!pip install langchain
!pip install langgraph
!pip install -U langmem
!pip install mcp
!pip install -qU "langchain[groq]"


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import MemorySaver
from langmem import create_manage_memory_tool , create_search_memory_tool


In [ ]:
#Long term memory management tools to store memory and search memory
#when you use MongoDB or Redis you replace this with DB schemas - which are nothing but JSON objects - Key Value Pairs
memory_tool =[
    create_manage_memory_tool(namespace = ("agent_memory",)),
    create_search_memory_tool(namespace = ("agent_memory_search",))
]

In [ ]:
import os , getpass
import dotenv
#setup a memory store
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
#in session memory what langchain offers
#if we want permanent memory across session we need to use a persistent memory store like Redis / Mongo-db etc ... refer code as below for that

store = InMemoryStore(
    index ={
        "dims":1536,
        "embed":"openai:text-embedding-3-small"
    }
)


In [ ]:
from langchain_core.tools import tool
from typing import List , Literal
@tool
def add(a:int,b:int) ->int:
  """Adds two integers and returns the result."""
  return a+b

In [ ]:
model = init_chat_model("qwen/qwen3-32b", api_key= GROQ_API_KEY ,model_provider="groq")
#add memory checkpointer
checkpointer = MemorySaver() #this checkpointer gets changed to Mongo or Redis when we want persistent store and this can be Async
#in case of marketing agent we did these  things
#1 -Used Redis as persistent store and in that mem0 for checkpointer
#2 -Used compressing the context ....
#3 -Used langmem to do so
#4 -tool call binding as well
#5 -search the memory
#activate react agent
#add the model and memory management tool into the agent - create and search memory
#*args and **kwargs are used to allow functions to accept an arbitrary number of arguments
agent = create_react_agent(model=model,tools = [add,*memory_tool],checkpointer=checkpointer,store=store)

In [ ]:
#start executing the agent
text = "Hi, Please create marketing plan for reducing customer acquistion cost. This plan is for CMO Exec Level so plan should be executable"
session_id = 1
result = agent.invoke({"messages":[{"role":"user","content":text}]},config ={"configurable":{"session_id":session_id, "thread_id": str(session_id)}} )
print(result["messages"][-1].content)

In [ ]:
#adding more text into the prompt
text ="also pls suggest campaigns"
session_id = 1
result = agent.invoke({"messages":[{"role":"user","content":text}]},config ={"configurable":{"session_id":session_id, "thread_id": str(session_id)}} )
print(result["messages"][-1].content)

In [ ]:
#now change the session_id and check memory
text = "generate marketing plan for increasing sales"
session_id = 2
result = agent.invoke({"messages":[{"role":"user","content":text}]},config ={"configurable":{"session_id":session_id, "thread_id": str(session_id)}} )
print(result["messages"][-1].content)

In [ ]:
#leveraging langmem
namespace = {"agent_memory","{user_id}"} # creataing a agent memory segregrated by user_id - This is what I did
text = "pls publish my marketing plan"
session_id = 2
user_id = "ab"
result = agent.invoke({"messages":[{"role":"user","content":text}]},config ={"configurable":{"session_id":session_id, "thread_id": user_id}} )

In [ ]:
items = store.search("agenr_memory",)
for item in items:
  print(item.namespace,item.value)

In [ ]:
!pip install fastapi
!pip install langgraph-checkpoint-sqlite


In [ ]:
#develop conv agent with memory and asyn using langgraph and langchain
import os
import uvicorn
from dotenv import load_dotenv
from typing import List, TypedDict , Annotated
from contextlib import asynccontextmanager

#Fast API Imports
from fastapi import  FastAPI , Request , Form
from fastapi.responses import HTMLResponse , RedirectResponse
from fastapi.templating import Jinja2Templates
from starlette import status

#import langchain /langgraph imports
from langchain_core.messages import AIMessage , HumanMessage , BaseMessage
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph , END

In [ ]:
#Langgraph state will have three things to store
#user prompt
#message history
#response from LLM

class ChatState(TypedDict):
  user_prompt:str
  messages:Annotated[List[BaseMessage],add_messages] #Annotated type(list) with metadata. #add_messages is a reducer
  response:str


In [ ]:
#define the virtual assistant

async def process_user_prompt_node(state:ChatState):
  user_message = state["user_prompt"]
  return {"messages":[HumanMessage(content=user_prompt)]}

async def call_model_node(state:ChatState):
  llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
  messages = state["messages"]
  if not messages or not isinstance(messages[-1],HumanMessage):
    return{"response":"no message generated by LLM"}
  try:
    #call llm for response
    response = await llm.ainvoke(messages) #async operation of calling LLM
    #return the LLM response
    return{"response":response.content}
  except Exception as e:
    return{"response":"sorry, error enountered no response from LLM"}

async def process_bot_response_node(state:ChatState):
  bot_response = state["response"]
  return {"messages":[AIMessage(content=bot_response)]}


In [ ]:
#FastAPI LifeSpan Function
@asynccontextmanager
async def lifespan(app:FastAPI):
  print("Starting up:Intializing Resources")

  async with AsyncSqliteSaver.from_conn_string(SQLITE_DATABASE_PATH) as checkpointer:
    print("AsyncSqliteSaver Connection Established")
    #intiate the graph
    workflow = StateGraph(ChatState) #intialize the graph with persistent chat state #THANKS A LOT GOD

    #add nodes
    workflow.add_node("process_user_prompt",process_user_prompt_node)
    workflow.add_node("call_model",call_model_node)
    workflow.add_node("generate_response",END)

    #add edges
    workflow.set_entry_point("process_user_prompt")
    workflow.add_edge("process_user_prompt","call_model")
    workflow.add_edge("call_model","generate_response")
    workflow.add_edge("generate_response",END)

    #compile the graph
    app.state.graph = workflow.compile(checkpointer = checkpointer) #this checkpointer can be Redis or Postgress depending upon the persistent store
    app.state.checkpointer = checkpointer
  yield
  print("Shutting down:closing resources")


In [ ]:
#helper function for memory management/ storing chats and retrieving those
async def get_chat_history_messages(request:Request,thread_id:str):
   #request is of object type FastAPI request
   config = {"configurable":{"thread_id":thread_id}}
   chat_history_messages = []
   app_graph = request.app.chat.graph
   try:
    current_state = await app_graph.aget_state(config)
    if current_state and current_state.values.get("messages"):
      chat_history_messages = current_state.values["messages"]
      print(f"loaded state in memory {len(chat_history_messages)} messages for session {thread_id}")
    else:
      print("no loaded state")
   except Exception as e:
      print("Exception")
   return chat_history_messages

In [ ]:
#intialize FastAPI App Intialization with LifeSpan

app = FastAPI(lifespan = lifespan)

#mount jinja templates
templates = Jinja2Templates(directory = "templates")

In [ ]:
#FAST API end points
SESSION_ID = 1
@app.get("/",response_class = HTMLResponse)
async def read_root(request:Request):
  """ Render chat page with Conversation History"""
  #get the chat history for the current session
  chat_history = await get_chat_history_messages(request,SESSION_ID)
  return templates.TemplateResponse("chat.html",{"request":request,"chat_history":chat_history})
@app.post("/chat")
async def chat_endpoint(request:Request,user_prompt:str = Form(...)):
  if not user_prompt:
    return RedirectResponse(url="/",status_code = status.HTTP_303_SEE_OTHER)
  graph_input = {"user_prompt":user_prompt}
  #create a configurable dictionary with the current session / thread ID
  config = {"configurable":{"thread_id":SESSION_ID}}
  app_graph = request.app.state.graph
  try:
    await app_graph.ainvoke(graph_input,config = config)
  except Exception as e:
    print(f"error invoking graph for session id {SESSION_ID} : {e}" )
  return RedirectResponse(url="/",status_code = status.HTTP_303_SEE_OTHER)


In [ ]:
#run the FastAPI app....
import nest_asyncio
import uvicorn
import asyncio

nest_asyncio.apply()

# Create a Uvicorn server configuration
config = uvicorn.Config("app:app", host="0.0.0.0", reload=False, loop="asyncio")

# Create a Uvicorn server instance
server = uvicorn.Server(config)

# Run the server in the existing event loop
# This will block until the server is stopped
asyncio.run(server.serve())

In [ ]:
#code for MCP
#why MCP - standardize away to access tools and give it to LLM
#agents can be exposed MCP - this can enable Agent Factories - Agent as a tool Pattern
#subagents can run in there own sandbox(E2B) and can complete there own task independently
#I used subagents for Code and Text2sql  run in there own sandbox - this helps for security
#Subagents chaining and comm is done thru message passing by the main agent / Orchestrator pattern ....
#Subagents are registered in a registry https://www.vladsnewsletter.com/p/sub-agents
#A2A communication can be thru Schema....
#Master Agent forces a schema on sub agents and sub agents respond back on that schema
#Agents can use VFS - Virtual File System , Todo-list , System prompt and sub agent

#MCP
"""
1. MCP Prompt - what protocol , what are tool inputs , return - text , image etc....
2. MCP Client
3. MCP Server
4. Resources
"""

#https://thenewstack.io/15-best-practices-for-building-mcp-servers-in-production/
#https://thenewstack.io/how-elicitation-in-mcp-brings-human-in-the-loop-to-ai-tools/

#Agents -> use MCP key factors:
"""
1.Use Aysnc
2.Use Error handling / time out from MCP server
3.OAuth Security
4.Elicitation of response from Human .... Human + MCP
5.use json-rpc or HTTPStreaming for faster response from MCP service
6.
"""


In [ ]:
#simple mcp server
!pip install fastmcp

In [ ]:
from fastmcp import FastMCPServer
from fastmcp import FastMCP

server = FastMCP("Demo")
@server.tool()
async def add(a:int,b:int) ->int:
  return a+b
@server.tool()
async def multiply(a:int,b:int) ->int:
  return a*b

@server.tool()
async def divide(a:int,b:int) ->int:
  return a/b

@server.tool()
async def subtract(a:int,b:int) ->int:
  return a-b

@server.tool()
async def greet_user_formal(name:str) ->str:

  """
  A tool that returns a greeting message in a very long tone
  Args:
  name:str
  Returns:str
  """
  return f"Hello {name}! How can I assist you today?"

@server.prompt
def greet_user_prompt(name:str) ->str:
  """ Generates  a message asking for a greeting"""
  return f"""
  Return a greeting message for a user called '{name}'
  if the user is called 'Laurent' , use a formal style , else use a street style
  """
if __name__ == "_main()__":
  server.run(transport="stdio")